# Qt Non-Blocking Integration with Jupyter

This notebook demonstrates how Qt can run in a non-blocking way within Jupyter notebooks using the `%gui qt` magic command.

In [4]:
# Enable Qt event loop integration
# This allows Qt GUIs to run without blocking the notebook
%gui qt

In [5]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QSlider, QLabel
from PyQt5.QtCore import Qt
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

In [6]:
# Global variable that can be modified from notebook cells
freq = 1.0

In [7]:
class SineWaveWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Sine Wave Demo - Qt Non-Blocking")
        self.setGeometry(100, 100, 800, 600)
        
        # Create central widget and layout
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout(central_widget)
        
        # Create frequency slider
        self.freq_label = QLabel(f"Frequency: {freq:.1f}")
        layout.addWidget(self.freq_label)
        
        self.freq_slider = QSlider(Qt.Horizontal)
        self.freq_slider.setMinimum(1)
        self.freq_slider.setMaximum(100)
        self.freq_slider.setValue(int(freq * 10))
        self.freq_slider.valueChanged.connect(self.on_slider_change)
        layout.addWidget(self.freq_slider)
        
        # Create matplotlib figure
        self.figure = Figure(figsize=(8, 4))
        self.canvas = FigureCanvas(self.figure)
        layout.addWidget(self.canvas)
        
        self.ax = self.figure.add_subplot(111)
        self.update_plot()
        
        # Timer to check for external frequency changes
        from PyQt5.QtCore import QTimer
        self.timer = QTimer()
        self.timer.timeout.connect(self.check_external_changes)
        self.timer.start(100)  # Check every 100ms
        
        self.last_freq = freq
    
    def on_slider_change(self, value):
        global freq
        freq = value / 10.0
        self.freq_label.setText(f"Frequency: {freq:.1f}")
        self.update_plot()
    
    def check_external_changes(self):
        """Check if freq was changed externally (from a notebook cell)"""
        global freq
        if abs(freq - self.last_freq) > 0.01:
            self.last_freq = freq
            self.freq_slider.setValue(int(freq * 10))
            self.freq_label.setText(f"Frequency: {freq:.1f}")
            self.update_plot()
    
    def update_plot(self):
        global freq
        self.ax.clear()
        x = np.linspace(0, 2*np.pi, 1000)
        y = np.sin(freq * x)
        self.ax.plot(x, y, 'b-', linewidth=2)
        self.ax.set_xlabel('x')
        self.ax.set_ylabel('sin(freq * x)')
        self.ax.set_title(f'Sine Wave (frequency = {freq:.1f})')
        self.ax.grid(True)
        self.canvas.draw()

In [9]:
# Create and show the window - it will run non-blocking!
# The Qt event loop is integrated with Jupyter's event loop via %gui qt
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)

window = SineWaveWindow()
window.show()

print("Qt window created and shown. You can continue executing cells!")

Qt window created and shown. You can continue executing cells!


## Test Non-Blocking Execution

The cells below demonstrate that the notebook remains responsive while the Qt GUI is running.

In [10]:
# This cell executes immediately, proving the GUI didn't block
print("This prints while the Qt window is still open!")
print(f"Current frequency: {freq}")

This prints while the Qt window is still open!
Current frequency: 1.0


In [11]:
# Change the frequency from the notebook - the GUI will update automatically!
freq = 5.0
print(f"Changed frequency to {freq} - watch the GUI update!")

Changed frequency to 5.0 - watch the GUI update!


In [12]:
# Try another value
freq = 0.5
print(f"Changed frequency to {freq}")

Changed frequency to 0.5


In [13]:
# You can do other computations while the GUI runs
import time
for i in range(5):
    print(f"Computing... {i+1}/5")
    time.sleep(0.5)
print("Done! GUI is still running.")

Computing... 1/5
Computing... 2/5
Computing... 3/5
Computing... 4/5
Computing... 5/5
Done! GUI is still running.


In [14]:
# Close the window programmatically
window.close()
print("Window closed.")

Window closed.


## How This Works

The `%gui qt` magic command integrates Qt's event loop with IPython/Jupyter's event loop. This means:

1. **Non-blocking**: Qt windows can stay open without blocking cell execution
2. **Bidirectional communication**: 
   - GUI can update variables (via slider)
   - Notebook cells can update GUI (by changing `freq`)
3. **Event processing**: Both Qt and Jupyter events are processed in an integrated loop

### Key Points for imgui_bundle:

- Qt achieves this through IPython's event loop integration system
- The GUI framework's event loop needs to yield control regularly
- For imgui_bundle, you're using `asyncio` with `await asyncio.sleep(0)` which is similar - it yields control to let Jupyter process other tasks
- The main difference: Qt uses IPython's built-in integration, while you're implementing manual async rendering